In [ ]:
import os,sys
import kfp

In [ ]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=token)

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-titanic-pl',
    description='sample titanic pipeline with dkube components'
)
def titanic_pipeline(token):
    train       = dkube_training_op(token, '{"image":"ocdr/d3-datascience-sklearn:v0.23.2-1"}',
                                    framework="sklearn", version="0.23.2",
                                    program="sklearn-examples", run_script="python titanic/train.py",
                                    datasets='["titanic"]', outputs='["titanic"]',
                                    input_dataset_mounts='["/titanic"]',
                                    output_mounts='["/model"]',
                                    envs='[{"EPOCHS": "1"}]')

    serving     = dkube_serving_op(token, train.outputs['artifact'], device='cpu', 
                                    serving_image='{"image":"ocdr/sklearnserver:0.23.2"}',
                                    transformer_image='{"image":"ocdr/d3-datascience-sklearn:v0.23.2-1"}',
                                    transformer_project="sklearn-examples",
                                    transformer_code='titanic/transformer.py').after(train)

In [ ]:
client.create_run_from_pipeline_func(titanic_pipeline, arguments={"token":token})

In [ ]:
#kfp.compiler.Compiler().compile(titanic_pipeline, "titanic_pipeline.zip")